# 目录
## 1. 模块导入
## 2. fashion mnist 数据集准备
## 3. 创建`keras`模型
## 4.  `save` 和 `save_weights`保存模型
  - ### 4.1 `save`保存(保存模型结构和参数)
    - `save`
    - `keras.models.load_model`
  - ### 4.2 `save_weights`保存（只保存参数）
    - `save_weights`
    - `load_weights`
  
## 5. callbacks 模型保存
  - `keras.callbacks.ModelCheckpoint`

## 1. 模块导入

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn

from tensorflow import keras
import tensorflow as tf
import sys
import os
import time
import datetime

for module in [np, pd, mpl, sklearn, keras, tf]:
    print(module.__name__, module.__version__)

numpy 1.18.1
pandas 0.25.3
matplotlib 3.1.2
sklearn 0.22.1
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf
tensorflow 2.1.0


## 2. fashion mnist 数据集准备

In [3]:
# 取出fashion mnist 数据集
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()

x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]


# 标准化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train_scaled = scaler.fit_transform(x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)
x_valid_scaled = scaler.transform(x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)
x_test_scaled = scaler.transform(x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)

print(x_train_scaled.shape, y_train.shape)
print(x_valid_scaled.shape, y_valid.shape)
print(x_test_scaled.shape, y_test.shape)


# 制作dataset 数据集
def make_dataset(images, labels, epochs, batch_size, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size).prefetch(50) # prefetch 先取出50个样本准备
    return dataset


batch_size = 64
epochs = None
train_dataset = make_dataset(x_train_scaled, y_train, epochs, batch_size)
valid_dataset = make_dataset(x_valid_scaled, y_valid, 1, batch_size, shuffle=False)
test_dataset = make_dataset(x_test_scaled, y_test, 1, batch_size, shuffle=False)

(55000, 28, 28, 1) (55000,)
(5000, 28, 28, 1) (5000,)
(10000, 28, 28, 1) (10000,)


## 3. 创建`keras`模型

In [4]:
def create_model():
    model = keras.models.Sequential([
        keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=(28, 28, 1)),
        keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"),
        keras.layers.MaxPool2D(pool_size=2),

        keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"),
        keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"),
        keras.layers.MaxPool2D(pool_size=2),

        keras.layers.Conv2D(filters=128, kernel_size=3, padding="same", activation="relu"),
        keras.layers.Conv2D(filters=128, kernel_size=3, padding="same", activation="relu"),
        keras.layers.MaxPool2D(pool_size=2),

        keras.layers.Flatten(),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dense(10, activation="softmax")
    ])

    model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.SGD(lr=0.01), metrics=["accuracy"])
    return model

## 4.  `save` 和 `save_weights`保存模型

  - ### 4.1 `save`保存(保存模型结构和参数)

建新模型训练

In [8]:
model = create_model()
model.fit(train_dataset, 
          steps_per_epoch = len(x_train_scaled)//batch_size,
          validation_data=valid_dataset, 
          validation_steps = len(x_valid_scaled)//batch_size,
          epochs=10)

Train for 859 steps, validate for 78 steps
Epoch 1/10
859/859 [==============================] - 5s 6ms/step - loss: 1.0429 - accuracy: 0.6245 - val_loss: 0.5318 - val_accuracy: 0.8121
Epoch 2/10
859/859 [==============================] - 5s 5ms/step - loss: 0.4865 - accuracy: 0.8211 - val_loss: 0.4446 - val_accuracy: 0.8421
Epoch 3/10
859/859 [==============================] - 5s 5ms/step - loss: 0.4051 - accuracy: 0.8506 - val_loss: 0.3611 - val_accuracy: 0.8690
Epoch 4/10
859/859 [==============================] - 5s 5ms/step - loss: 0.3638 - accuracy: 0.8659 - val_loss: 0.3645 - val_accuracy: 0.8660
Epoch 5/10
859/859 [==============================] - 5s 5ms/step - loss: 0.3360 - accuracy: 0.8761 - val_loss: 0.3172 - val_accuracy: 0.8832
Epoch 6/10
859/859 [==============================] - 5s 5ms/step - loss: 0.3145 - accuracy: 0.8844 - val_loss: 0.3028 - val_accuracy: 0.8880
Epoch 7/10
859/859 [==============================] - 5s 5ms/step - loss: 0.2975 - accuracy: 0.8898 - val

验证

In [10]:
model.evaluate(test_dataset)

157/157 [==============================] - 0s 3ms/step - loss: 0.3306 - accuracy: 0.8747


[0.33064243088292466, 0.8747]

`save`保存模型

In [12]:
model_save_dir = "keras_hdf5_model/save"
if not os.path.exists(model_save_dir):
    os.makedirs(model_save_dir)

model_save_file = os.path.join(model_save_dir, "fashion_mnist.h5")
model.save(model_save_file)

加载模型，测试

In [13]:
new_model = keras.models.load_model(model_save_file)
new_model.evaluate(test_dataset)

157/157 [==============================] - 0s 3ms/step - loss: 0.3306 - accuracy: 0.8747


[0.33064243088292466, 0.8747]

加载的模型还能训练

In [14]:
new_model.fit(train_dataset, 
          steps_per_epoch = len(x_train_scaled)//batch_size,
          validation_data=valid_dataset, 
          validation_steps = len(x_valid_scaled)//batch_size,
          epochs=10)

Train for 859 steps, validate for 78 steps
Epoch 1/10
859/859 [==============================] - 5s 6ms/step - loss: 0.2496 - accuracy: 0.9077 - val_loss: 0.2807 - val_accuracy: 0.8944
Epoch 2/10
859/859 [==============================] - 5s 5ms/step - loss: 0.2380 - accuracy: 0.9121 - val_loss: 0.2706 - val_accuracy: 0.9012
Epoch 3/10
859/859 [==============================] - 5s 5ms/step - loss: 0.2292 - accuracy: 0.9151 - val_loss: 0.2547 - val_accuracy: 0.9048
Epoch 4/10
859/859 [==============================] - 5s 5ms/step - loss: 0.2209 - accuracy: 0.9187 - val_loss: 0.2801 - val_accuracy: 0.8982
Epoch 5/10
859/859 [==============================] - 5s 5ms/step - loss: 0.2125 - accuracy: 0.9215 - val_loss: 0.2553 - val_accuracy: 0.9058
Epoch 6/10
859/859 [==============================] - 5s 5ms/step - loss: 0.2053 - accuracy: 0.9233 - val_loss: 0.2519 - val_accuracy: 0.9107
Epoch 7/10
859/859 [==============================] - 5s 5ms/step - loss: 0.1971 - accuracy: 0.9265 - val

  - ### 4.2 `save_weights`保存（只保存参数）

保存模型参数

In [16]:
model_save_dir = "keras_hdf5_model/save_weights"
if not os.path.exists(model_save_dir):
    os.makedirs(model_save_dir)

model_save_file = os.path.join(model_save_dir, "fashion_mnist_weight.h5")
model.save_weights(model_save_file)

加载模型参数

In [18]:
new_model02 = create_model()
new_model02.load_weights(model_save_file)

测试

In [19]:
new_model02.evaluate(test_dataset)

157/157 [==============================] - 1s 3ms/step - loss: 0.3306 - accuracy: 0.8747


[0.33064243088292466, 0.8747]

还可以再训练

In [20]:
new_model02.fit(train_dataset, 
          steps_per_epoch = len(x_train_scaled)//batch_size,
          validation_data=valid_dataset, 
          validation_steps = len(x_valid_scaled)//batch_size,
          epochs=10)

Train for 859 steps, validate for 78 steps
Epoch 1/10
859/859 [==============================] - 5s 6ms/step - loss: 0.2485 - accuracy: 0.9080 - val_loss: 0.2783 - val_accuracy: 0.8940
Epoch 2/10
859/859 [==============================] - 5s 5ms/step - loss: 0.2388 - accuracy: 0.9119 - val_loss: 0.2565 - val_accuracy: 0.9058
Epoch 3/10
859/859 [==============================] - 5s 5ms/step - loss: 0.2295 - accuracy: 0.9142 - val_loss: 0.2657 - val_accuracy: 0.9002
Epoch 4/10
859/859 [==============================] - 5s 5ms/step - loss: 0.2213 - accuracy: 0.9175 - val_loss: 0.2486 - val_accuracy: 0.9046
Epoch 5/10
859/859 [==============================] - 5s 5ms/step - loss: 0.2118 - accuracy: 0.9216 - val_loss: 0.2642 - val_accuracy: 0.9067
Epoch 6/10
859/859 [==============================] - 5s 5ms/step - loss: 0.2051 - accuracy: 0.9245 - val_loss: 0.2617 - val_accuracy: 0.9062
Epoch 7/10
859/859 [==============================] - 5s 5ms/step - loss: 0.1974 - accuracy: 0.9273 - val

## 5. callbacks 模型保存

In [23]:
log_dir = "keras_hdf5_model/log"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

model_dir = "keras_hdf5_model/callbacks"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
model_file = os.path.join(model_dir, "fashion_mnist_callback.h5")

callbacks = [
    keras.callbacks.TensorBoard(log_dir),
    keras.callbacks.ModelCheckpoint(
        model_file,
        save_best_only=False,
        save_weights_only=False  # 保存模型和参数,  True时，和上面的save_weights模型一样；False时，和上面的save一样
    ),
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3)
]

In [24]:
model = create_model()
model.fit(train_dataset, 
          steps_per_epoch = len(x_train_scaled)//batch_size,
          validation_data=valid_dataset, 
          validation_steps = len(x_valid_scaled)//batch_size,
          callbacks=callbacks,
          epochs=10)

Train for 859 steps, validate for 78 steps
Epoch 1/10
859/859 [==============================] - 6s 7ms/step - loss: 0.9529 - accuracy: 0.6647 - val_loss: 0.5050 - val_accuracy: 0.8251
Epoch 2/10
859/859 [==============================] - 5s 6ms/step - loss: 0.4712 - accuracy: 0.8243 - val_loss: 0.4007 - val_accuracy: 0.8558
Epoch 3/10
859/859 [==============================] - 5s 6ms/step - loss: 0.3969 - accuracy: 0.8533 - val_loss: 0.3615 - val_accuracy: 0.8682
Epoch 4/10
859/859 [==============================] - 5s 6ms/step - loss: 0.3575 - accuracy: 0.8677 - val_loss: 0.3469 - val_accuracy: 0.8746
Epoch 5/10
859/859 [==============================] - 5s 6ms/step - loss: 0.3323 - accuracy: 0.8783 - val_loss: 0.3276 - val_accuracy: 0.8818
Epoch 6/10
859/859 [==============================] - 5s 6ms/step - loss: 0.3105 - accuracy: 0.8857 - val_loss: 0.3169 - val_accuracy: 0.8846
Epoch 7/10
859/859 [==============================] - 5s 6ms/step - loss: 0.2959 - accuracy: 0.8911 - val

In [25]:
model.evaluate(test_dataset)

157/157 [==============================] - 0s 3ms/step - loss: 0.3124 - accuracy: 0.8859


[0.31237847073252795, 0.8859]

In [27]:
new_model = keras.models.load_model(model_file)
model.evaluate(test_dataset)

157/157 [==============================] - 0s 2ms/step - loss: 0.3124 - accuracy: 0.8859


[0.31237847073252795, 0.8859]